<a href="https://colab.research.google.com/github/bythyag/BTP_Project/blob/main/simulation_and_dataset_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from collections import defaultdict

# Set random seed
np.random.seed(42) # You can change this seed value to any integer

# Constants
NUM_COMPONENTS = 4
NUM_BATCHES = 4  # Changed to 4 as requested
OUTCOME_TYPES = ['repair', 'replace', 'non']
TIME_RANGE = (1, 10) # Time range in minutes

def generate_time():
    return round(np.random.uniform(TIME_RANGE[0], TIME_RANGE[1]), 2)

def simulate_component(component_id):
    process_time = generate_time()
    outcome = np.random.choice(OUTCOME_TYPES)
    if outcome == 'repair':
        repair_time = generate_time()
        total_time = process_time + repair_time
        return f"C{component_id}_repair", process_time, repair_time, total_time
    elif outcome == 'replace':
        replace_time = generate_time()
        total_time = process_time + replace_time
        return f"C{component_id}_replace", process_time, replace_time, total_time
    else:
        return f"C{component_id}_non", process_time, 0, process_time

def simulate_batch():
    batch_pathway = []
    batch_time = 0
    component_details = []
    for component_id in range(1, NUM_COMPONENTS + 1):
        outcome, process_time, additional_time, total_time = simulate_component(component_id)
        batch_pathway.append(outcome)
        batch_time += total_time
        component_details.append({
            'component': f'C{component_id}',
            'outcome': outcome,
            'process_time': process_time,
            'additional_time': additional_time,
            'total_time': total_time
        })
    return '_'.join(batch_pathway), batch_time, component_details

def simulate_full_run():
    full_pathway = []
    total_time = 0
    all_batch_details = []
    for batch_id in range(1, NUM_BATCHES + 1):
        batch_pathway, batch_time, component_details = simulate_batch()
        full_pathway.append(f"B{batch_id}_{batch_pathway}")
        total_time += batch_time
        all_batch_details.append({
            'batch_id': batch_id,
            'pathway': batch_pathway,
            'time': batch_time,
            'components': component_details
        })
    return {
        'full_pathway': '_'.join(full_pathway),
        'total_time': total_time,
        'batch_details': all_batch_details
    }

def generate_dataset(num_simulations):
    dataset = [simulate_full_run() for _ in range(num_simulations)]
    return dataset

# Generate dataset
num_simulations = 10000
dataset = generate_dataset(num_simulations)

# Print some sample results
for i, sample in enumerate(dataset[:2], 1):
    print(f"Simulation {i}:")
    print(f"Full Pathway: {sample['full_pathway']}")
    print(f"Total Time: {sample['total_time']:.2f} minutes")
    print("Batch Details:")
    for batch in sample['batch_details']:
        print(f" Batch {batch['batch_id']}:")
        print(f" Pathway: {batch['pathway']}")
        print(f" Time: {batch['time']:.2f} minutes")
        for component in batch['components']:
            print(f" {component['component']} ({component['outcome']}):")
            print(f" Process Time: {component['process_time']:.2f} min")
            print(f" Additional Time: {component['additional_time']:.2f} min")
            print(f" Total Time: {component['total_time']:.2f} min")
    print()

# Calculate and print statistics
pathway_counts = defaultdict(int)
total_times = []
for sample in dataset:
    pathway_counts[sample['full_pathway']] += 1
    total_times.append(sample['total_time'])

print(f"Total number of simulations: {num_simulations}")
print(f"Number of unique full pathways: {len(pathway_counts)}")
print(f"\nAverage total time: {np.mean(total_times):.2f} minutes")
print(f"Minimum total time: {min(total_times):.2f} minutes")
print(f"Maximum total time: {max(total_times):.2f} minutes")
print("\nTop 5 most common full pathways:")
for pathway, count in sorted(pathway_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{pathway}: {count} occurrences")

Simulation 1:
Full Pathway: B1_C1_repair_C2_repair_C3_non_C4_repair_B2_C1_replace_C2_replace_C3_repair_C4_repair_B3_C1_non_C2_non_C3_non_C4_non_B4_C1_repair_C2_repair_C3_repair_C4_repair
Total Time: 134.17 minutes
Batch Details:
 Batch 1:
 Pathway: C1_repair_C2_repair_C3_non_C4_repair
 Time: 31.38 minutes
 C1 (C1_repair):
 Process Time: 4.37 min
 Additional Time: 2.65 min
 Total Time: 7.02 min
 C2 (C2_repair):
 Process Time: 8.02 min
 Additional Time: 2.40 min
 Total Time: 10.42 min
 C3 (C3_non):
 Process Time: 2.40 min
 Additional Time: 0.00 min
 Total Time: 2.40 min
 C4 (C4_repair):
 Process Time: 5.13 min
 Additional Time: 6.41 min
 Total Time: 11.54 min
 Batch 2:
 Pathway: C1_replace_C2_replace_C3_repair_C4_repair
 Time: 39.88 minutes
 C1 (C1_replace):
 Process Time: 7.37 min
 Additional Time: 1.51 min
 Total Time: 8.88 min
 C2 (C2_replace):
 Process Time: 7.50 min
 Additional Time: 2.91 min
 Total Time: 10.41 min
 C3 (C3_repair):
 Process Time: 2.64 min
 Additional Time: 6.56 min


In [ ]:
import numpy as np
from collections import defaultdict

# Constants
NUM_COMPONENTS = 4
NUM_BATCHES = 4
OUTCOME_TYPES = ['repair', 'replace', 'non']
TIME_RANGE = (1, 10)  # Time range in minutes

def generate_time():
    return round(np.random.uniform(TIME_RANGE[0], TIME_RANGE[1]), 2)

def simulate_component(component_id):
    process_time = generate_time()
    outcome = np.random.choice(OUTCOME_TYPES)

    if outcome == 'repair':
        repair_time = generate_time()
        total_time = process_time + repair_time
        return f"C{component_id}_repair", process_time, repair_time, total_time
    elif outcome == 'replace':
        replace_time = generate_time()
        total_time = process_time + replace_time
        return f"C{component_id}_replace", process_time, replace_time, total_time
    else:
        return f"C{component_id}_non", process_time, 0, process_time

def simulate_batch():
    batch_pathway = []
    batch_time = 0
    component_details = []

    for component_id in range(1, NUM_COMPONENTS + 1):
        outcome, process_time, additional_time, total_time = simulate_component(component_id)
        batch_pathway.append(outcome)
        batch_time += total_time
        component_details.append({
            'component': f'C{component_id}',
            'outcome': outcome,
            'process_time': process_time,
            'additional_time': additional_time,
            'total_time': total_time
        })

    return '_'.join(batch_pathway), batch_time, component_details

def simulate_full_run():
    full_pathway = []
    total_time = 0
    all_batch_details = []

    for batch_id in range(1, NUM_BATCHES + 1):
        batch_pathway, batch_time, component_details = simulate_batch()
        full_pathway.append(f"B{batch_id}_{batch_pathway}")
        total_time += batch_time
        all_batch_details.append({
            'batch_id': batch_id,
            'pathway': batch_pathway,
            'time': batch_time,
            'components': component_details
        })

    return {
        'full_pathway': '_'.join(full_pathway),
        'total_time': total_time,
        'batch_details': all_batch_details
    }

def generate_dataset(num_simulations):
    dataset = [simulate_full_run() for _ in range(num_simulations)]
    return dataset

# Generate dataset
num_simulations = 10000
dataset = generate_dataset(num_simulations)

# Print some sample results
for i, sample in enumerate(dataset[:2], 1):
    print(f"Simulation {i}:")
    print(f"Full Pathway: {sample['full_pathway']}")
    print(f"Total Time: {sample['total_time']:.2f} minutes")
    print("Batch Details:")
    for batch in sample['batch_details']:
        print(f"  Batch {batch['batch_id']}:")
        print(f"    Pathway: {batch['pathway']}")
        print(f"    Time: {batch['time']:.2f} minutes")
        for component in batch['components']:
            print(f"      {component['component']} ({component['outcome']}):")
            print(f"        Process Time: {component['process_time']:.2f} min")
            print(f"        Additional Time: {component['additional_time']:.2f} min")
            print(f"        Total Time: {component['total_time']:.2f} min")
    print()

# Calculate and print statistics
pathway_counts = defaultdict(int)
total_times = []

for sample in dataset:
    pathway_counts[sample['full_pathway']] += 1
    total_times.append(sample['total_time'])

print(f"Total number of simulations: {num_simulations}")
print(f"Number of unique full pathways: {len(pathway_counts)}")
print(f"\nAverage total time: {np.mean(total_times):.2f} minutes")
print(f"Minimum total time: {min(total_times):.2f} minutes")
print(f"Maximum total time: {max(total_times):.2f} minutes")
print("\nTop 5 most common full pathways:")
for pathway, count in sorted(pathway_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{pathway}: {count} occurrences")

Simulation 1:
Full Pathway: B1_C1_replace_C2_non_C3_repair_C4_repair_B2_C1_replace_C2_repair_C3_non_C4_repair_B3_C1_replace_C2_replace_C3_non_C4_repair_B4_C1_replace_C2_replace_C3_replace_C4_replace
Total Time: 153.76 minutes
Batch Details:
  Batch 1:
    Pathway: C1_replace_C2_non_C3_repair_C4_repair
    Time: 24.04 minutes
      C1 (C1_replace):
        Process Time: 2.24 min
        Additional Time: 4.33 min
        Total Time: 6.57 min
      C2 (C2_non):
        Process Time: 1.09 min
        Additional Time: 0.00 min
        Total Time: 1.09 min
      C3 (C3_repair):
        Process Time: 3.49 min
        Additional Time: 2.57 min
        Total Time: 6.06 min
      C4 (C4_repair):
        Process Time: 1.26 min
        Additional Time: 9.06 min
        Total Time: 10.32 min
  Batch 2:
    Pathway: C1_replace_C2_repair_C3_non_C4_repair
    Time: 42.59 minutes
      C1 (C1_replace):
        Process Time: 3.66 min
        Additional Time: 3.88 min
        Total Time: 7.54 min
      C

In [ ]:
import numpy as np
from collections import defaultdict

# Set random seed
np.random.seed(42)  # You can change this seed value to any integer

# Constants
NUM_COMPONENTS = 4
NUM_BATCHES = 4
OUTCOME_TYPES = ['repair', 'replace', 'non']
TIME_RANGE = (1, 10)  # Time range in minutes

def generate_time():
    return round(np.random.uniform(TIME_RANGE[0], TIME_RANGE[1]), 2)

def simulate_component(component_id):
    process_time = generate_time()
    outcome = np.random.choice(OUTCOME_TYPES)
    if outcome == 'repair':
        repair_time = generate_time()
        total_time = process_time + repair_time
        return f"C{component_id}_repair", process_time, repair_time, total_time
    elif outcome == 'replace':
        replace_time = generate_time()
        total_time = process_time + replace_time
        return f"C{component_id}_replace", process_time, replace_time, total_time
    else:
        return f"C{component_id}_non", process_time, 0, process_time

def simulate_batch():
    batch_pathway = []
    batch_time = 0
    component_details = []
    for component_id in range(1, NUM_COMPONENTS + 1):
        outcome, process_time, additional_time, total_time = simulate_component(component_id)
        batch_pathway.append(outcome)
        batch_time += total_time
        component_details.append({
            'component': f'C{component_id}',
            'outcome': outcome,
            'process_time': process_time,
            'additional_time': additional_time,
            'total_time': total_time
        })
    return '_'.join(batch_pathway), batch_time, component_details

def simulate_full_run():
    full_pathway = []
    total_time = 0
    all_batch_details = []
    for batch_id in range(1, NUM_BATCHES + 1):
        batch_pathway, batch_time, component_details = simulate_batch()
        full_pathway.append(f"B{batch_id}_{batch_pathway}")
        total_time += batch_time
        all_batch_details.append({
            'batch_id': batch_id,
            'pathway': batch_pathway,
            'time': batch_time,
            'components': component_details
        })
    return {
        'full_pathway': '_'.join(full_pathway),
        'total_time': total_time,
        'batch_details': all_batch_details
    }

def generate_dataset(num_simulations):
    dataset = [simulate_full_run() for _ in range(num_simulations)]
    return dataset

# Generate dataset
num_simulations = 10000
dataset = generate_dataset(num_simulations)

# Print some sample results
for i, sample in enumerate(dataset[:2], 1):
    print(f"Simulation {i}:")
    print(f"Full Pathway: {sample['full_pathway']}")
    print(f"Total Time: {sample['total_time']:.2f} minutes")
    print("Batch Details:")
    for batch in sample['batch_details']:
        print(f" Batch {batch['batch_id']}:")
        print(f" Pathway: {batch['pathway']}")
        print(f" Time: {batch['time']:.2f} minutes")
        for component in batch['components']:
            print(f" {component['component']} ({component['outcome']}):")
            print(f" Process Time: {component['process_time']:.2f} min")
            print(f" Additional Time: {component['additional_time']:.2f} min")
            print(f" Total Time: {component['total_time']:.2f} min")
    print()

# Calculate and print statistics
pathway_counts = defaultdict(int)
total_times = []
for sample in dataset:
    pathway_counts[sample['full_pathway']] += 1
    total_times.append(sample['total_time'])

print(f"Total number of simulations: {num_simulations}")
print(f"Number of unique full pathways: {len(pathway_counts)}")
print(f"\nAverage total time: {np.mean(total_times):.2f} minutes")
print(f"Minimum total time: {min(total_times):.2f} minutes")
print(f"Maximum total time: {max(total_times):.2f} minutes")
print("\nTop 5 most common full pathways:")
for pathway, count in sorted(pathway_counts.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{pathway}: {count} occurrences")

Simulation 1:
Full Pathway: B1_C1_repair_C2_repair_C3_non_C4_repair_B2_C1_replace_C2_replace_C3_repair_C4_repair_B3_C1_non_C2_non_C3_non_C4_non_B4_C1_repair_C2_repair_C3_repair_C4_repair
Total Time: 134.17 minutes
Batch Details:
 Batch 1:
 Pathway: C1_repair_C2_repair_C3_non_C4_repair
 Time: 31.38 minutes
 C1 (C1_repair):
 Process Time: 4.37 min
 Additional Time: 2.65 min
 Total Time: 7.02 min
 C2 (C2_repair):
 Process Time: 8.02 min
 Additional Time: 2.40 min
 Total Time: 10.42 min
 C3 (C3_non):
 Process Time: 2.40 min
 Additional Time: 0.00 min
 Total Time: 2.40 min
 C4 (C4_repair):
 Process Time: 5.13 min
 Additional Time: 6.41 min
 Total Time: 11.54 min
 Batch 2:
 Pathway: C1_replace_C2_replace_C3_repair_C4_repair
 Time: 39.88 minutes
 C1 (C1_replace):
 Process Time: 7.37 min
 Additional Time: 1.51 min
 Total Time: 8.88 min
 C2 (C2_replace):
 Process Time: 7.50 min
 Additional Time: 2.91 min
 Total Time: 10.41 min
 C3 (C3_repair):
 Process Time: 2.64 min
 Additional Time: 6.56 min


In [ ]:
import random
from typing import List, Tuple, Dict
import numpy as np
import csv
from datetime import datetime

class Component:
    def __init__(self, name: str):
        self.name = name

    def process(self) -> float:
        return max(0, np.random.normal(10, 2))  # Mean: 10, Std Dev: 2

    def fails(self) -> bool:
        return random.choice([True, False])

    def repair_or_replace(self) -> Tuple[str, float]:
        action = random.choice(["repair", "replace"])
        location = random.choice(["in-house", "outside"])
        time = random.uniform(1, 5)  # Random time between 1 and 5 hours
        return f"{action}_{location}", time

class Batch:
    def __init__(self, name: str):
        self.name = name

def run_simulation(components: List[Component], batches: int) -> List[Dict]:
    simulation_data = []

    for batch_num in range(batches):
        batch = Batch(f"Batch_{batch_num + 1}")

        for component in components:
            step_data = {
                "Batch": batch.name,
                "Component": component.name,
                "Process Time": component.process(),
                "Failure": "No",
                "Failure Action": "N/A",
                "Failure Time": 0
            }

            if component.fails():
                action, repair_time = component.repair_or_replace()
                step_data["Failure"] = "Yes"
                step_data["Failure Action"] = action
                step_data["Failure Time"] = repair_time
                step_data["Process Time"] += repair_time

            simulation_data.append(step_data)

    return simulation_data

def save_results(all_simulations: List[List[Dict]], filename: str):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ["Simulation", "Batch", "Component", "Process Time", "Failure", "Failure Action", "Failure Time", "Cumulative Time"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for sim_num, simulation in enumerate(all_simulations, 1):
            cumulative_time = 0
            for step in simulation:
                cumulative_time += step["Process Time"]
                row = {
                    "Simulation": sim_num,
                    "Batch": step["Batch"],
                    "Component": step["Component"],
                    "Process Time": f"{step['Process Time']:.2f}",
                    "Failure": step["Failure"],
                    "Failure Action": step["Failure Action"],
                    "Failure Time": f"{step['Failure Time']:.2f}",
                    "Cumulative Time": f"{cumulative_time:.2f}"
                }
                writer.writerow(row)

def run_multiple_simulations(num_components: int, num_batches: int, num_simulations: int) -> List[List[Dict]]:
    all_simulations = []

    for sim in range(num_simulations):
        components = [Component(f"Component_{i+1}") for i in range(num_components)]
        simulation_data = run_simulation(components, num_batches)
        all_simulations.append(simulation_data)
        total_time = sum(step["Process Time"] for step in simulation_data)
        print(f"Simulation {sim + 1}: Total processing time: {total_time:.2f} hours")

    return all_simulations

# Main execution
num_components = int(input("Enter the number of components: "))
num_batches = int(input("Enter the number of batches: "))
num_simulations = int(input("Enter the number of simulations to run: "))

all_results = run_multiple_simulations(num_components, num_batches, num_simulations)

# Generate a unique filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"detailed_simulation_results_{timestamp}.csv"

save_results(all_results, filename)

# Calculate summary statistics
total_times = [sum(step["Process Time"] for step in simulation) for simulation in all_results]
avg_time = sum(total_times) / num_simulations
min_time = min(total_times)
max_time = max(total_times)

print(f"\nSimulation Summary:")
print(f"Number of simulations: {num_simulations}")
print(f"Average processing time: {avg_time:.2f} hours")
print(f"Minimum processing time: {min_time:.2f} hours")
print(f"Maximum processing time: {max_time:.2f} hours")
print(f"Detailed results saved to: {filename}")

# These variables will be available for inspection after running the script
simulation_results = all_results
simulation_average = avg_time
simulation_min = min_time
simulation_max = max_time

Enter the number of components: 2
Enter the number of batches: 1
Enter the number of simulations to run: 100
Simulation 1: Total processing time: 30.80 hours
Simulation 2: Total processing time: 19.42 hours
Simulation 3: Total processing time: 25.86 hours
Simulation 4: Total processing time: 22.61 hours
Simulation 5: Total processing time: 29.32 hours
Simulation 6: Total processing time: 21.41 hours
Simulation 7: Total processing time: 25.27 hours
Simulation 8: Total processing time: 20.40 hours
Simulation 9: Total processing time: 20.84 hours
Simulation 10: Total processing time: 27.06 hours
Simulation 11: Total processing time: 24.54 hours
Simulation 12: Total processing time: 25.58 hours
Simulation 13: Total processing time: 17.97 hours
Simulation 14: Total processing time: 28.07 hours
Simulation 15: Total processing time: 24.60 hours
Simulation 16: Total processing time: 15.65 hours
Simulation 17: Total processing time: 21.56 hours
Simulation 18: Total processing time: 20.37 hours


In [4]:
import numpy as np
import json
import random

# User inputs (you can modify these as needed)
num_simulations = 1       # Number of simulations to run (e.g., 1)
num_total_batches = 100   # Total number of batch passes in each simulation
batches_per_pass = 2      # Number of batches processed each time
num_components = 2        # Number of machine components

processing_time_mean = 10  # Mean processing time for each machine
processing_time_std = 2    # Standard deviation of processing time
weibull_params = {
    'component_1': {'beta': 1.5, 'eta': 100},
    'component_2': {'beta': 2.0, 'eta': 150}
}
restoration_factors = {
    'repair': 0.5,       # Restoration factor after repair (Kijima model)
    'replacement': 0.0   # Restoration factor after replacement (new component)
}

# Function to calculate failure time using the Weibull distribution
def calculate_failure_time(beta, eta, age):
    """Calculate the next failure time using the Weibull distribution."""
    u = np.random.uniform(0, 1)
    time_to_failure = eta * ((-np.log(1 - u)) ** (1 / beta))
    return time_to_failure + age

# Function to calculate new virtual age after repair or replacement
def restoration(virtual_age, restoration_factor):
    """Calculate the new virtual age after repair or replacement."""
    return virtual_age * restoration_factor

# Initialize the simulation data storage
simulation_data = []

# Run the simulations
for sim in range(num_simulations):
    sim_record = {'simulation': sim + 1, 'batch_passes': []}

    # Initialize the state of each component for this simulation
    components = {}
    for i in range(1, num_components + 1):
        components[f'component_{i}'] = {
            'virtual_age': 0,
            'weibull_beta': weibull_params[f'component_{i}']['beta'],
            'weibull_eta': weibull_params[f'component_{i}']['eta'],
            'next_failure_time': calculate_failure_time(
                weibull_params[f'component_{i}']['beta'],
                weibull_params[f'component_{i}']['eta'],
                0  # Initial age is 0
            )
        }

    # Process the total number of batch passes
    for batch_pass_num in range(num_total_batches):
        batch_pass_record = {'batch_pass_number': batch_pass_num + 1, 'batches': []}
        for batch_num in range(batches_per_pass):
            batch_record = {'batch_number': batch_num + 1, 'components': []}
            for comp_name, comp_data in components.items():
                component_record = {'component': comp_name}

                # Processing time for the component
                processing_time = np.random.normal(processing_time_mean, processing_time_std)
                component_record['processing_time'] = processing_time

                # Update virtual age with processing time
                comp_data['virtual_age'] += processing_time

                # Check if the component fails during this processing time
                if comp_data['virtual_age'] >= comp_data['next_failure_time']:
                    component_record['status'] = 'Failed'

                    # Decide between repair and replacement based on virtual age
                    eta = comp_data['weibull_eta']
                    age = comp_data['virtual_age']
                    decision = 'repair' if age < (0.8 * eta) else 'replacement'
                    component_record['decision'] = decision

                    # Decide between in-house and external repairman randomly
                    repairman = random.choice(['in-house', 'external'])
                    component_record['repairman'] = repairman

                    # Random repair/replacement time
                    if decision == 'repair':
                        repair_time = np.random.uniform(5, 15) if repairman == 'in-house' else np.random.uniform(10, 20)
                    else:  # Replacement
                        repair_time = np.random.uniform(20, 30) if repairman == 'in-house' else np.random.uniform(30, 40)
                    component_record['repair_time'] = repair_time

                    # Add repair/replacement time to processing time
                    processing_time += repair_time
                    component_record['total_processing_time'] = processing_time

                    # Update virtual age based on restoration factor
                    rf = restoration_factors[decision]
                    comp_data['virtual_age'] = restoration(comp_data['virtual_age'], rf)

                    # Schedule next failure time
                    beta = comp_data['weibull_beta']
                    eta = comp_data['weibull_eta']
                    comp_data['next_failure_time'] = calculate_failure_time(beta, eta, comp_data['virtual_age'])

                else:
                    component_record['status'] = 'Passed'
                    component_record['total_processing_time'] = processing_time

                # Record current virtual age
                component_record['virtual_age'] = comp_data['virtual_age']
                batch_record['components'].append(component_record)
            batch_pass_record['batches'].append(batch_record)
        sim_record['batch_passes'].append(batch_pass_record)
    simulation_data.append(sim_record)

# Save the simulation data to a JSON file
with open('simulation_results.json', 'w') as json_file:
    json.dump(simulation_data, json_file, indent=4)

print("Simulation complete. Results saved to 'simulation_results.json'.")


Simulation complete. Results saved to 'simulation_results.json'.


In [5]:
import random
import math
import json
from typing import List, Dict, Tuple

class Component:
    def __init__(self, name: str, weibull_params: Tuple[float, float]):
        self.name = name
        self.age = 0
        self.virtual_age = 0
        self.weibull_shape, self.weibull_scale = weibull_params
        self.restoration_factor = random.uniform(0.6, 0.9)  # Random restoration factor between 0.6 and 0.9
        self.repair_count = 0

    def process_time(self) -> float:
        return max(0, random.gauss(10, 2))  # Normal distribution with mean 10 and std dev 2

    def fails(self) -> bool:
        failure_prob = 1 - math.exp(-(self.virtual_age / self.weibull_scale) ** self.weibull_shape)
        return random.random() < failure_prob

    def repair(self, is_replacement: bool) -> float:
        if is_replacement:
            repair_time = random.uniform(6, 12)  # Replacement takes longer
            self.age = 0
            self.virtual_age = 0
            self.repair_count = 0
            self.restoration_factor = random.uniform(0.6, 0.9)  # New component, new restoration factor
        else:
            repair_time = random.uniform(2, 5) if random.choice([True, False]) else random.uniform(4, 8)
            self.virtual_age = self.age * (1 - self.restoration_factor)
            self.repair_count += 1
        return repair_time

    def decide_replacement(self) -> bool:
        age_factor = self.age / self.weibull_scale
        repair_factor = self.repair_count / 5  # Assume 5 repairs is a lot
        restoration_efficiency = 1 - self.restoration_factor  # Lower restoration factor means less efficient repairs
        random_factor = random.random()

        replacement_score = 0.3 * age_factor + 0.3 * repair_factor + 0.3 * restoration_efficiency + 0.1 * random_factor
        return replacement_score > 0.7  # 70% threshold for replacement

class Batch:
    def __init__(self, size: int):
        self.size = size

def process_batches(components: List[Component], batch_size: int) -> Dict:
    batch_data = []
    total_time = 0

    for _ in range(2):  # Process 2 batches
        batch = Batch(batch_size)
        current_batch_data = {"size": batch.size, "components": []}

        for component in components:
            component_data = {
                "name": component.name,
                "initial_age": component.age,
                "initial_virtual_age": component.virtual_age,
                "restoration_factor": component.restoration_factor
            }

            process_time = component.process_time() * batch.size
            total_time += process_time

            if component.fails():
                is_replacement = component.decide_replacement()
                repair_time = component.repair(is_replacement)
                total_time += repair_time
                component_data.update({
                    "failed": True,
                    "action": "replaced" if is_replacement else "repaired",
                    "repair_time": repair_time
                })
            else:
                component_data["failed"] = False

            component.age += process_time
            component.virtual_age += process_time

            component_data.update({
                "process_time": process_time,
                "final_age": component.age,
                "final_virtual_age": component.virtual_age,
                "repair_count": component.repair_count,
                "final_restoration_factor": component.restoration_factor
            })

            current_batch_data["components"].append(component_data)

        batch_data.append(current_batch_data)

    return {"batches": batch_data, "total_time": total_time}

def run_simulation(num_components: int, batch_size: int, num_simulations: int, sub_simulations: int,
                   weibull_params: Dict[str, Tuple[float, float]] = None) -> List[Dict]:
    if weibull_params is None:
        weibull_params = {f"Component_{i}": (random.uniform(1.5, 2.5), random.uniform(100, 200))
                          for i in range(num_components)}

    simulation_data = []

    for sim in range(num_simulations):
        components = [Component(f"Component_{i}", weibull_params[f"Component_{i}"])
                      for i in range(num_components)]

        sim_data = {"simulation": sim, "sub_simulations": []}

        for sub_sim in range(sub_simulations):
            sub_sim_data = process_batches(components, batch_size)
            sub_sim_data["sub_simulation"] = sub_sim
            sim_data["sub_simulations"].append(sub_sim_data)

        simulation_data.append(sim_data)

    return simulation_data

def save_to_json(data: List[Dict], filename: str = "simulation_results.json"):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)

def main():
    num_components = int(input("Enter the number of components: "))
    batch_size = int(input("Enter the batch size: "))
    num_simulations = int(input("Enter the number of simulations: "))
    sub_simulations = int(input("Enter the number of sub-simulations: "))

    use_custom_weibull = input("Do you want to input custom Weibull parameters? (y/n): ").lower() == 'y'

    if use_custom_weibull:
        weibull_params = {}
        for i in range(num_components):
            shape = float(input(f"Enter Weibull shape parameter for Component_{i}: "))
            scale = float(input(f"Enter Weibull scale parameter for Component_{i}: "))
            weibull_params[f"Component_{i}"] = (shape, scale)
    else:
        weibull_params = None

    simulation_data = run_simulation(num_components, batch_size, num_simulations, sub_simulations, weibull_params)
    save_to_json(simulation_data)
    print(f"Simulation complete. Results saved to 'simulation_results.json'")

if __name__ == "__main__":
    main()

Enter the number of components: 2
Enter the batch size: 2
Enter the number of simulations: 1
Enter the number of sub-simulations: 100
Do you want to input custom Weibull parameters? (y/n): n
Simulation complete. Results saved to 'simulation_results.json'
